In [1]:
get_ipython().magic(u'matplotlib inline')
import numpy
import timeit
import glob
import cv2
import os
import re
import pickle
import matplotlib.pyplot as plt
from skimage import transform
import skimage
from skimage import io

In [2]:
import sklearn
from sklearn.model_selection import train_test_split   ### import sklearn tool

import keras
from keras.preprocessing import image as image_utils
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [3]:
# function to load video from file

def load_set(videofile):
    all_frames = []
    vidcap = cv2.VideoCapture(videofile)
    success,image = vidcap.read()
    count = 0       
    error = ''      
    success = True  

    img = []        
    while success: 
        success, img = vidcap.read()  
        count += 1  
        frames = []  
        for j in range(0,99):
            try:
                success, img = vidcap.read()
                tmp = skimage.color.rgb2gray(numpy.array(img))
                tmp = skimage.transform.downscale_local_mean(tmp, (5,5))
                frames.append(tmp)
                count+=99
            except:
                count+=1
    
        if numpy.shape(frames)==(99, 144, 256):
            all_frames.append(frames)
        elif numpy.shape(frames[0])==(144,256):
            all_frames.append(numpy.concatenate((all_frames[-1][-(99-len(frames)):], frames)))
        elif numpy.shape(frames[0])!=(144,256):
            error = 'Video is of incorrect resolution.'
    vidcap.release()
    del frames; del image
    return all_frames, error

In [4]:
img_filepath = '' 
neg_all = glob.glob(img_filepath + 'negative/*.mp4')               
pos_all = glob.glob(img_filepath + 'positive/*.mp4')                 

all_files = (pos_all + neg_all)
print(len(neg_all), len(pos_all))
print(len(all_files))

def label_matrix(values):
    n_values = numpy.max(values) + 1    
    return numpy.eye(n_values)[values]

labels = numpy.concatenate(([1]*len(pos_all), [0]*len(neg_all[0:len(pos_all)])))
labels = label_matrix(labels)


3 4
7


In [21]:
def make_dataset(rand):
    seq1 = numpy.zeros((len(rand), 99, 144, 256))
    for i,fi in enumerate(rand):                    
        print (i, fi)                               
        if fi[-4:] == '.mp4':
            t = load_set(fi)                        
        if np.ndim(t) == (99,144,256):              
            seq1[i] = t
        else:# TypeError
            'Image has shape ', np.ndim(t), 'but needs to be shape', np.ndim(seq1[0])        
    print (np.ndim(seq1))
    return seq1

In [6]:
from array import array
import numpy as np

#only testing
x_testA = np.asarray(all_files)
y_testA = np.asarray(labels)

In [7]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, TimeDistributed
from keras.layers import LSTM

batch_size = 15
num_classes = 2
epochs = 30
row_hidden = 128
col_hidden = 128
frame, row, col = (99, 144, 256)

x = Input(shape=(frame, row, col))
encoded_rows = TimeDistributed(LSTM(row_hidden))(x)  
encoded_columns = LSTM(col_hidden)(encoded_rows)

### set up prediction and compile the model
prediction = Dense(num_classes, activation='softmax')(encoded_columns)
model = Model(x, prediction)


0 positive/positive.mp4
1 positive/positive1.mp4
2 positive/positive3.mp4
3 positive/positive2.mp4
4 negative/negative.mp4
5 negative/negative2.mp4
6 negative/negative1.mp4
4
